In [15]:
# data
import os

from nltk.corpus import stopwords

path_root = "C:\\BCSpace\\study\\ThisSemester\\IR\\project\\workspace"
dir_source = "test_data"
dir_temp = "_temp"  # folder name for the processed files
fn_indexer = 'indexer.csv'    # new file to store the indexer
fn_doc_id = 'doc_id.csv'
fn_doc_list = 'doc_list.csv'
fn_doc_len = 'doc_len.csv'

path_source = os.path.join(path_root, dir_source)
path_temp = os.path.join(path_root, dir_temp)
doc_id = {}
doc_list = []
doc_len = {}

stop_words = {'i', 'me', 'my', 'myself', 'we', 
              'our', 'ours', 'ourselves', 'you', 
              "you're", "you've", "you'll", "you'd", 
              'your', 'yours', 'yourself', 'yourselves', 
              'he', 'him', 'his', 'himself', 'she', 
              "she's", 'her', 'hers', 'herself', 'it', 
              "it's", 'its', 'itself', 'they', 'them', 
              'their', 'theirs', 'themselves', 'what', 
              'which', 'who', 'whom', 'this', 'that', 
              "that'll", 'these', 'those', 'am', 'is', 
              'are', 'was', 'were', 'be', 'been', 
              'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 
              'the', 'and', 'but', 'if', 'or', 'because', 
              'as', 'until', 'while', 'of', 'at', 'by', 
              'for', 'with', 'about', 'against', 'between', 
              'into', 'through', 'during', 'before', 'after', 
              'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 
              'again', 'further', 'then', 'once', 'here', 
              'there', 'when', 'where', 'why', 'how', 
              'all', 'any', 'both', 'each', 'few',
              'more', 'most', 'other', 'some', 'such', 
              'no', 'nor', 'not', 'only', 'own', 
              'same', 'so', 'than', 'too', 'very', 
              's', 't', 'can', 'will', 'just', 'don', 
              "don't", 'should', "should've", 'now', 'd', 
              'll', 'm', 'o', 're', 've', 'y', 'ain', 
              'aren', "aren't", 'couldn', "couldn't", 
              'didn', "didn't", 'doesn', "doesn't", 'hadn', 
              "hadn't", 'hasn', "hasn't", 'haven', "haven't", 
              'isn', "isn't", 'ma', 'mightn', "mightn't", 
              'mustn', "mustn't", 'needn', "needn't", 'shan', 
              "shan't", 'shouldn', "shouldn't", 'wasn', 
              "wasn't", 'weren', "weren't", 'won', "won't", 
              'wouldn', "wouldn't"}
# stop_words = set(stopwords.words('english'))
stop_words.add('s')


In [16]:
# Some functions
import csv
from math import log10

from nltk import RegexpTokenizer
import sys


# Recursively finds size of objects
def get_size(obj, seen=None):
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size


# tokenize

def get_term_freq(textlines):
    title = textlines[0]
    text = textlines[7]
    text = text.lower()
    # word_tokenizer = RegexpTokenizer(r'\w+|[^\w\s]')
    word_tokenizer = RegexpTokenizer('[A-Za-z]+')
    words_title = word_tokenizer.tokenize(title)
    term_set = set(words_title).difference(stop_words)
    term_freq = {}
    for w in words_title:
        if w in term_set:
            if w in term_freq:
                term_freq[w] = term_freq[w] + 1
            else:
                term_freq[w] = 1

    words_text = word_tokenizer.tokenize(text)
    term_set = set(words_text).difference(stop_words)
    for w in words_text:
        if w in term_set:
            if w in term_freq:
                term_freq[w] = term_freq[w] + 5
            else:
                term_freq[w] = 10
    return term_freq, len(words_title) + len(words_text) * 5


# store sub dictionary in .csv

In [17]:
# Calculate the indexer and store into 'indexer.csv'

# get the sub dictionaries
from sys import getsizeof
from math import sqrt

chunk_size = 15000000



def process_directions():
    chunk_num = 0
    term_index = {}
    term_index.clear()
    doc_list.clear()
    doc_id.clear()
    doc_len.clear()
    if not os.path.exists(path_temp):
        os.mkdir(path_temp)
    for root, dirs, files in os.walk(path_source):
        for file_name in files:
            fpn = os.path.join(root, file_name)
            doc_list.append(fpn)
            file_cnt = len(doc_list) - 1
            doc_id[fpn] = file_cnt

            with open(fpn, mode='r', errors='ignore', encoding='utf-8') as f:
                text = f.readlines()
            term_freq, d_len = get_term_freq(text)
            doc_len[file_cnt] = d_len
            for term, value in term_freq.items():
                if term in term_index:
                    term_index[term].append((file_cnt, value))
                else:
                    term_index[term] = [(file_cnt, value)]
            if file_cnt % 1000 == 0:
                print(file_cnt)
    return term_index

indexer = process_directions()
print('fin')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
fin


In [18]:
def dump():
    with open(os.path.join(path_temp, fn_indexer), mode='w+', newline='', encoding='utf-8') as f:
        f_writer = csv.writer(f)
        f_writer.writerows(indexer.items())
    with open(os.path.join(path_temp, fn_doc_id), mode='w+', newline='', encoding='utf-8') as f:
        f_writer = csv.writer(f)
        f_writer.writerows(doc_id.items())
    with open(os.path.join(path_temp, fn_doc_len), mode='w+', newline='', encoding='utf-8') as f:
        f_writer = csv.writer(f)
        f_writer.writerows(doc_len.items())
    with open(os.path.join(path_temp, fn_doc_list), mode='w+', newline='', encoding='utf-8') as f:
        f_writer = csv.writer(f)
        f_writer.writerow(doc_list)
    print('dump fin')


# todo: load data
def load():

    print('load fin')

dump()

dump fin


In [23]:
from math import log

def weighted_tf(tf, d_len, avg_len):
    return 2 * tf / (0.25 + 0.75 * d_len / avg_len + tf)

def idf(df, n):
    if df == 0:
        return 0
    return log(n / df, 10)

def hw3_bm25_11812420(query, doc_len = {}, dictionary = {}):
    avg_doc_len = 0
    doc_num = len(doc_len)
    for dl in doc_len:
        avg_doc_len = avg_doc_len + dl / doc_num

    candidate = {}
    for k in query:
        df = len(dictionary[k])
        for did, tf in dictionary[k]:
            if did not in candidate.keys():
                candidate[did] = 0
            candidate[did] = candidate[did] + weighted_tf(tf, doc_len[did], avg_doc_len) * idf(df, doc_num)

    candidate = sorted(candidate.items(), key= lambda x: (x[1]), reverse= True)
    return candidate[:10]

# Query
# for v in indexer['russia']:
#     print(doc_list[v[0]][len(path_root):], v[1])
query = ['china', 'japan', 'russia']
#
answer = hw3_bm25_11812420(query, doc_len, indexer)
print(answer)
for fid, weight in answer:
    print('======================')
    print(str(fid) + ';' + doc_list[fid] + ';' + str(weight))
    with open(doc_list[fid], mode='r', encoding='utf-8') as f:
        text = f.readlines()
    print('title: ' + text[0][:-1])
    print('date : ' + text[3])







[(3518, 6.661794433355222), (7170, 6.523378963330472), (3517, 6.488150612849635), (8209, 6.44053696953877), (4972, 6.334251051979649), (7091, 6.326927079297601), (2745, 6.322680762080596), (7442, 6.321386129754794), (8487, 6.314221718186095), (4598, 6.312473236562228)]
3518;C:\BCSpace\study\ThisSemester\IR\project\workspace\test_data\articles1\21186.csv;6.661794433355222
title: Kohei Uchimura Leads Japan to Gymnastics Gold, Dethroning China - The New York Times
date : 2017-02-03

7170;C:\BCSpace\study\ThisSemester\IR\project\workspace\test_data\articles1\25698.csv;6.523378963330472
title: Without Obama, Shinzo Abe’s Approach to U.S.-Japan Ties May Be Tested - The New York Times
date : 2017-01-05

3517;C:\BCSpace\study\ThisSemester\IR\project\workspace\test_data\articles1\21185.csv;6.488150612849635
title: Rio Olympics: Drama in the Pool and a Super Tuesday - The New York Times
date : 2016-08-18

8209;C:\BCSpace\study\ThisSemester\IR\project\workspace\test_data\articles1\26945.csv;6.440